# 第03章：LCEL基础（LangChain Expression Language）

## 本章学习目标

1. 理解LCEL的概念和重要性
2. 掌握Runnable接口和协议
3. 学会使用Pipe操作符（|）组合组件
4. 掌握@chain装饰器创建自定义Runnable
5. 理解RunnableLambda的作用
6. 学习数据传递和流式处理模式
7. 构建复杂的处理链

## 核心概念

- **LCEL**：LangChain Expression Language，LangChain的核心语法
- **Runnable**：LangChain中所有组件的标准接口
- **Pipe操作符（|）**：用于连接和组合不同组件
- **@chain装饰器**：创建自定义Runnable的标准方式
- **RunnableLambda**：将普通函数转换为Runnable

## 重要说明

LCEL是LangChain v1的核心，让组件组合变得简洁而强大。本章将系统学习这个强大的语法。


In [ ]:
# 环境配置
import os
import sys

_project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(_project_root)

from config import config
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 初始化模型
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    api_key=config.OPENAI_API_KEY,
    base_url=config.OPENAI_BASE_URL,
)

print("环境配置完成")
print("LCEL学习开始！")

环境配置完成
LCEL学习开始！


## 1. 什么是LCEL？

**LCEL (LangChain Expression Language)**是LangChain的核心语法，它让你能够：

### 核心特点

| 特点 | 说明 | 优势 |
|------|------|------|
| **统一接口** | 所有组件都实现Runnable接口 | 一致的使用方式 |
| **可组合性** | 使用Pipe操作符（`\|`）连接组件 | 简洁的代码 |
| **流式支持** | 自动支持流式处理 | 更好的用户体验 |
| **并行执行** | 支持batch批量处理 | 更高的性能 |
| **标准化** | invoke、stream、batch等标准方法 | 易于学习和使用 |

### 为什么重要？

在第02章中，我们这样调用模型：
```python
messages = prompt.format_messages(topic="AI")
response = model.invoke(messages)
```

使用LCEL，我们可以这样：
```python
chain = prompt | model
response = chain.invoke({"topic": "AI"})
```

**更简洁、更强大、更易维护！**


In [2]:
# 验证不同组件都是Runnable
from langchain_core.runnables import Runnable

# 创建组件
prompt = ChatPromptTemplate.from_template("给我讲一个关于{topic}的笑话")

print("【验证Runnable接口】")
print(f"ChatOpenAI是Runnable: {isinstance(model, Runnable)}")
print(f"ChatPromptTemplate是Runnable: {isinstance(prompt, Runnable)}")

print("\n【查看标准方法】")
runnable_methods = ['invoke', 'stream', 'batch']
for method in runnable_methods:
    has_model = hasattr(model, method)
    has_prompt = hasattr(prompt, method)
    print(f"  {method}(): Model={has_model}, Prompt={has_prompt}")

print("\n【测试Prompt的Runnable方法】")
# Prompt也可以直接invoke
formatted_messages = prompt.invoke({"topic": "程序员"})
print(f"Prompt.invoke()返回: {type(formatted_messages)}")

【验证Runnable接口】
ChatOpenAI是Runnable: True
ChatPromptTemplate是Runnable: True

【查看标准方法】
  invoke(): Model=True, Prompt=True
  stream(): Model=True, Prompt=True
  batch(): Model=True, Prompt=True

【测试Prompt的Runnable方法】
Prompt.invoke()返回: <class 'langchain_core.prompt_values.ChatPromptValue'>


In [3]:
# 第一个LCEL Chain
prompt = ChatPromptTemplate.from_template("请用{language}语言说'{message}'")

# 使用Pipe操作符创建Chain
chain = prompt | model

print("【第一个LCEL Chain】")
print("Chain类型:", type(chain))
print("Chain是Runnable:", isinstance(chain, Runnable))

# 调用Chain
result = chain.invoke({
    "language": "法语", 
    "message": "你好，世界"
})

print(f"\n输入: language='法语', message='你好，世界'")
print(f"输出: {result.content}")

print("\n" + "=" * 60)

# Chain也支持所有Runnable方法
print("\n【Chain的流式输出】")
print("AI回复（流式）：", end="", flush=True)
for chunk in chain.stream({"language": "日语", "message": "谢谢"}):
    print(chunk.content, end="", flush=True)
print()

【第一个LCEL Chain】
Chain类型: <class 'langchain_core.runnables.base.RunnableSequence'>
Chain是Runnable: True

输入: language='法语', message='你好，世界'
输出: 你好，世界用法语可以说成“Bonjour, le monde”。


【Chain的流式输出】
AI回复（流式）：用日语说“谢谢”是「ありがとう」(arigatou)。


In [4]:
from langchain_core.runnables import chain as chain_decorator

# 使用@chain装饰器创建自定义处理函数
@chain_decorator
def format_response(ai_message):
    """格式化AI回复，添加装饰"""
    content = ai_message.content
    formatted = f"""
╔══════════════════════════════════════╗
║              AI 回复                 ║
╠══════════════════════════════════════╣
║ {content:^36} ║
╚══════════════════════════════════════╝
"""
    return formatted.strip()

print("【@chain装饰器示例】")
print(f"format_response是Runnable: {isinstance(format_response, Runnable)}")

# 测试自定义函数
from langchain_core.messages import AIMessage
test_message = AIMessage(content="你好！")
result = format_response.invoke(test_message)
print(f"\n格式化结果:\n{result}")

print("\n" + "=" * 60)

# 将自定义函数加入Chain
extended_chain = prompt | model | format_response

print("\n【扩展的Chain】")
formatted_result = extended_chain.invoke({
    "language": "中文", 
    "message": "学习LangChain很有趣"
})
print(formatted_result)

【@chain装饰器示例】
format_response是Runnable: True

格式化结果:
╔══════════════════════════════════════╗
║              AI 回复                 ║
╠══════════════════════════════════════╣
║                 你好！                  ║
╚══════════════════════════════════════╝


【扩展的Chain】
╔══════════════════════════════════════╗
║              AI 回复                 ║
╠══════════════════════════════════════╣
║           学习LangChain很有趣。            ║
╚══════════════════════════════════════╝


In [ ]:
# Chain的批量处理
chain = prompt | model

print("【Chain的批量处理】")

inputs = [
    {"language": "英语", "message": "早上好"},
    {"language": "西班牙语", "message": "晚安"},
    {"language": "德语", "message": "很高兴认识你"},
]

# 批量调用
responses = chain.batch(inputs)

for i, (input_data, response) in enumerate(zip(inputs, responses), 1):
    print(f"\n请求 {i}:")
    print(f"  输入: {input_data}")
    print(f"  输出: {response.content}")

【Chain的批量处理】

请求 1:
  输入: {'language': '英语', 'message': '早上好'}
  输出: '早上好' in English is 'Good morning.'

请求 2:
  输入: {'language': '西班牙语', 'message': '晚安'}
  输出: '晚安'用西班牙语说是"Buenas noches"。

请求 3:
  输入: {'language': '德语', 'message': '很高兴认识你'}
  输出: 很高兴认识你用德语可以说：“Freut mich, dich kennenzulernen.”


## 3. Pipe操作符（|）- LCEL的核心

**Pipe操作符（`|`）**是LCEL最重要的语法，用于连接Runnable组件。

### 基本语法

```python
# 基本模式
chain = component1 | component2 | component3

# 具体例子
chain = prompt | model
```

### 工作原理

1. **数据流向**：数据从左到右流动
2. **自动适配**：前一个组件的输出自动成为下一个组件的输入
3. **类型安全**：LangChain会验证组件之间的兼容性

### 优势

- **简洁性**：一行代码完成复杂的组合
- **可读性**：清晰表达数据处理流程
- **可维护性**：容易添加、删除或替换组件

## 4. RunnableLambda - 函数式编程

**RunnableLambda**将普通函数自动转换为Runnable。实际上，@chain装饰器内部就是使用RunnableLambda。

### 两种创建方式

1. **使用@chain装饰器**（推荐）
2. **直接使用RunnableLambda**

### 使用场景

- **数据转换**：格式化、清理、转换数据
- **条件逻辑**：根据输入内容选择不同处理
- **外部API调用**：集成第三方服务
- **自定义处理**：任何你需要的自定义逻辑

In [6]:
from langchain_core.runnables import RunnableLambda

# 方式1：使用RunnableLambda（显式）
def extract_keywords(ai_message):
    """提取关键词"""
    content = ai_message.content.lower()
    
    # 简单的关键词提取（实际项目中可能使用NLP库）
    keywords = []
    key_phrases = ["python", "langchain", "ai", "模型", "学习", "编程", "开发"]
    
    for phrase in key_phrases:
        if phrase in content:
            keywords.append(phrase)
    
    return {
        "原文": ai_message.content,
        "关键词": keywords,
        "关键词数量": len(keywords)
    }

# 显式创建RunnableLambda
keyword_extractor = RunnableLambda(extract_keywords)

print("【RunnableLambda示例】")
print(f"keyword_extractor是Runnable: {isinstance(keyword_extractor, Runnable)}")

# 创建关键词提取Chain
keyword_chain = prompt | model | keyword_extractor

result = keyword_chain.invoke({
    "language": "中文",
    "message": "我正在学习Python和LangChain开发AI应用"
})

print("\n关键词提取结果:")
for key, value in result.items():
    print(f"  {key}: {value}")

print("\n" + "=" * 60)

# 方式2：使用@chain装饰器（推荐）
@chain_decorator
def sentiment_analyzer(ai_message):
    """情感分析器"""
    content = ai_message.content.lower()
    
    positive_words = ["好", "棒", "喜欢", "开心", "满意", "excellent", "great", "good"]
    negative_words = ["坏", "差", "不喜欢", "难过", "糟糕", "bad", "terrible", "awful"]
    
    positive_count = sum(1 for word in positive_words if word in content)
    negative_count = sum(1 for word in negative_words if word in content)
    
    if positive_count > negative_count:
        sentiment = "积极"
        score = positive_count - negative_count
    elif negative_count > positive_count:
        sentiment = "消极"  
        score = negative_count - positive_count
    else:
        sentiment = "中性"
        score = 0
    
    return {
        "情感": sentiment,
        "分数": score,
        "积极词数": positive_count,
        "消极词数": negative_count
    }

# 创建情感分析Chain
sentiment_chain = prompt | model | sentiment_analyzer

print("\n【@chain装饰器对比】")
sentiment_result = sentiment_chain.invoke({
    "language": "中文", 
    "message": "这个课程真的很好，我很喜欢！"
})

print("情感分析结果:")
for key, value in sentiment_result.items():
    print(f"  {key}: {value}")

【RunnableLambda示例】
keyword_extractor是Runnable: True

关键词提取结果:
  原文: 我正在学习Python和LangChain开发AI应用。
  关键词: ['python', 'langchain', 'ai', '学习', '开发']
  关键词数量: 5


【@chain装饰器对比】
情感分析结果:
  情感: 积极
  分数: 2
  积极词数: 2
  消极词数: 0


## 5. 复杂Chain组合

LCEL的真正威力在于组合复杂的处理流程。

### 组合模式

```python
# 简单线性Chain
chain = prompt | model | processor

# 复杂处理Chain  
complex_chain = (
    input_processor 
    | prompt 
    | model 
    | output_formatter 
    | validator
)
```

### 设计原则

1. **单一职责**：每个组件专注做一件事
2. **可组合**：组件之间松耦合
3. **可测试**：每个组件都可以独立测试
4. **可复用**：组件可以在不同Chain中复用


In [7]:
# 构建复杂的文本处理Chain

@chain_decorator
def input_validator(input_data):
    """输入验证器"""
    if not input_data.get("text"):
        raise ValueError("输入文本不能为空")
    
    text = input_data["text"].strip()
    if len(text) < 2:
        raise ValueError("输入文本太短")
    
    return {"text": text, "original_length": len(input_data["text"])}

@chain_decorator
def text_preprocessor(data):
    """文本预处理器"""
    text = data["text"]
    
    # 清理和标准化
    cleaned_text = text.replace("\n", " ").replace("\t", " ")
    while "  " in cleaned_text:
        cleaned_text = cleaned_text.replace("  ", " ")
    
    return {
        **data,
        "cleaned_text": cleaned_text,
        "word_count": len(cleaned_text.split())
    }

@chain_decorator
def create_analysis_prompt(data):
    """创建分析提示词"""
    text = data["cleaned_text"]
    
    prompt_template = ChatPromptTemplate.from_template("""
请对以下文本进行全面分析：

文本：{text}

请提供：
1. 主题概括
2. 情感倾向
3. 语言风格
4. 关键信息

分析：""")
    
    return prompt_template.format_messages(text=text)

@chain_decorator
def result_formatter(ai_response):
    """结果格式化器"""
    analysis = ai_response.content
    
    return {
        "时间戳": "2024-01-15 10:30:00",
        "分析结果": analysis,
        "状态": "分析完成",
        "字符数": len(analysis)
    }

# 组合成复杂Chain
text_analysis_chain = (
    input_validator 
    | text_preprocessor 
    | create_analysis_prompt 
    | model 
    | result_formatter
)

print("【复杂Chain示例】")

# 测试复杂Chain
test_input = {
    "text": """
    LangChain是一个强大的框架，它让开发者能够轻松构建基于语言模型的应用。
    通过LCEL，我们可以用简洁的语法组合复杂的处理流程。
    这真是太棒了！
    """
}

try:
    final_result = text_analysis_chain.invoke(test_input)
    
    print("分析完成！")
    for key, value in final_result.items():
        if key == "分析结果":
            print(f"{key}:\n{value}")
        else:
            print(f"{key}: {value}")
            
except Exception as e:
    print(f"处理出错: {e}")

【复杂Chain示例】
分析完成！
时间戳: 2024-01-15 10:30:00
分析结果:
好的，以下是对该文本的全面分析：

### 1. 主题概括
文本的主题围绕LangChain框架展开，强调了其在构建语言模型应用方面的强大能力。提到的LCEL（LangChain Expression Language）提供了一种简洁的语法，使得开发者能够组合复杂的处理流程，从而提高开发效率。

### 2. 情感倾向
文本整体呈现出积极的情感倾向。使用了“强大”、“轻松”、“简洁”、“太棒了”等正面词汇，表达了对LangChain及其功能的高度赞赏和认可。

### 3. 语言风格
文本使用了简洁明了的语言风格，直截了当地传达了信息。句子结构简单，易于理解，适合技术类内容的表达。此外，使用了感叹句（“这真是太棒了！”），增加了情感的表达，使得内容更加生动。

### 4. 关键信息
- **LangChain**：一个框架，专注于构建基于语言模型的应用。
- **LCEL**：LangChain Expression Language，提供简洁的语法用于组合复杂处理流程。
- **开发者受益**：强调了框架的易用性和高效性，使开发者能够更轻松地进行开发。

综上所述，该文本简洁地介绍了LangChain框架及其语言处理能力，展现了积极的情感倾向，并采用了易懂的语言风格。
状态: 分析完成
字符数: 544


## 6. 数据传递模式

在复杂的Chain中，理解数据如何在组件之间传递是关键。

### 传递规则

1. **自动映射**：前一个组件的输出自动成为下一个组件的输入
2. **类型适配**：LangChain自动处理类型转换
3. **数据保持**：除非显式修改，否则数据会传递下去

### 常见模式

| 模式 | 说明 | 示例 |
|------|------|------|
| **直通传递** | 数据不变，直接传给下一个组件 | `dict -> dict` |
| **格式转换** | 改变数据格式但保持内容 | `str -> Messages` |
| **数据增强** | 添加额外信息 | `{a: 1} -> {a: 1, b: 2}` |
| **数据提取** | 从复杂对象中提取特定字段 | `AIMessage -> str` |


In [ ]:
# 数据传递模式示例

@chain_decorator
def step1_add_metadata(input_data):
    """步骤1：添加元数据"""
    print(f"Step 1 输入: {input_data} (类型: {type(input_data)})")
    
    result = {
        "original_input": input_data,
        "timestamp": "2024-01-15",
        "step": 1
    }
    print(f"Step 1 输出: {result}")
    return result

@chain_decorator  
def step2_process_data(data):
    """步骤2：处理数据"""
    print(f"\nStep 2 输入: {data} (类型: {type(data)})")
    
    result = {
        **data,  # 保持原有数据
        "processed": True,
        "step": 2,
        "message_count": len(str(data["original_input"]))
    }
    print(f"Step 2 输出: {result}")
    return result

@chain_decorator
def step3_extract_info(data):
    """步骤3：提取信息"""
    print(f"\nStep 3 输入: {data} (类型: {type(data)})")
    
    # 只提取部分信息
    result = f"处理完成！原始输入：{data['original_input']}，消息长度：{data['message_count']}"
    print(f"Step 3 输出: {result} (类型: {type(result)})")
    return result

# 创建数据传递Chain
data_flow_chain = step1_add_metadata | step2_process_data | step3_extract_info

print("【数据传递模式演示】")
final_result = data_flow_chain.invoke("Hello LCEL!")

print(f"\n最终结果: {final_result}")
print(f"最终类型: {type(final_result)}")

【数据传递模式演示】
Step 1 输入: Hello LCEL! (类型: <class 'str'>)
Step 1 输出: {'original_input': 'Hello LCEL!', 'timestamp': '2024-01-15', 'step': 1}

Step 2 输入: {'original_input': 'Hello LCEL!', 'timestamp': '2024-01-15', 'step': 1} (类型: <class 'dict'>)
Step 2 输出: {'original_input': 'Hello LCEL!', 'timestamp': '2024-01-15', 'step': 2, 'processed': True, 'message_count': 11}

Step 3 输入: {'original_input': 'Hello LCEL!', 'timestamp': '2024-01-15', 'step': 2, 'processed': True, 'message_count': 11} (类型: <class 'dict'>)
Step 3 输出: 处理完成！原始输入：Hello LCEL!，消息长度：11 (类型: <class 'str'>)

最终结果: 处理完成！原始输入：Hello LCEL!，消息长度：11
最终类型: <class 'str'>


## 7. 错误处理和调试

在复杂Chain中，错误处理和调试非常重要。

### 调试策略

1. **分步测试**：单独测试每个组件
2. **日志记录**：在关键步骤添加打印语句
3. **类型检查**：验证输入输出类型
4. **异常捕获**：优雅处理错误

### 最佳实践

- **明确接口**：清楚定义每个组件的输入输出
- **边界检查**：验证输入数据的有效性
- **渐进式构建**：逐步添加组件，而不是一次性构建复杂Chain
- **单元测试**：为每个组件编写测试

In [9]:
# 错误处理和调试示例

@chain_decorator
def safe_input_processor(data):
    """安全的输入处理器"""
    try:
        print(f"处理输入: {data}")
        
        # 输入验证
        if not isinstance(data, dict):
            raise TypeError(f"期望dict类型，得到{type(data)}")
        
        if "text" not in data:
            raise KeyError("缺少必需的'text'字段")
        
        text = data["text"]
        if not isinstance(text, str):
            raise TypeError("'text'字段必须是字符串")
        
        if len(text.strip()) == 0:
            raise ValueError("'text'字段不能为空")
        
        # 处理成功
        result = {
            "text": text.strip(),
            "length": len(text.strip()),
            "status": "success"
        }
        
        print(f"处理成功: {result}")
        return result
        
    except Exception as e:
        print(f"处理失败: {e}")
        # 返回错误信息而不是抛出异常
        return {
            "text": "",
            "length": 0, 
            "status": "error",
            "error": str(e)
        }

@chain_decorator
def error_aware_formatter(data):
    """错误感知的格式化器"""
    if data["status"] == "error":
        return f"❌ 处理出错: {data['error']}"
    
    return f"✅ 处理成功: '{data['text']}' (长度: {data['length']})"

# 创建带错误处理的Chain
safe_chain = safe_input_processor | error_aware_formatter

print("【错误处理示例】")

# 测试正常输入
print("\n测试1 - 正常输入:")
result1 = safe_chain.invoke({"text": "Hello LCEL"})
print(f"结果: {result1}")

# 测试错误输入
print("\n测试2 - 缺少字段:")
result2 = safe_chain.invoke({"message": "Hello"})  # 错误：缺少text字段
print(f"结果: {result2}")

print("\n测试3 - 空文本:")
result3 = safe_chain.invoke({"text": "   "})  # 错误：空文本
print(f"结果: {result3}")

print("\n测试4 - 错误类型:")
result4 = safe_chain.invoke("Hello")  # 错误：不是dict
print(f"结果: {result4}")

【错误处理示例】

测试1 - 正常输入:
处理输入: {'text': 'Hello LCEL'}
处理成功: {'text': 'Hello LCEL', 'length': 10, 'status': 'success'}
结果: ✅ 处理成功: 'Hello LCEL' (长度: 10)

测试2 - 缺少字段:
处理输入: {'message': 'Hello'}
处理失败: "缺少必需的'text'字段"
结果: ❌ 处理出错: "缺少必需的'text'字段"

测试3 - 空文本:
处理输入: {'text': '   '}
处理失败: 'text'字段不能为空
结果: ❌ 处理出错: 'text'字段不能为空

测试4 - 错误类型:
处理输入: Hello
处理失败: 期望dict类型，得到<class 'str'>
结果: ❌ 处理出错: 期望dict类型，得到<class 'str'>


## 8. 实战练习：构建智能摘要生成器

结合本章所学，构建一个完整的文档摘要生成Chain。

**需求分析：**
- 输入：长文档文本
- 处理：清理、分析、生成摘要
- 输出：格式化的摘要报告

**技术要点：**
- 使用@chain创建自定义组件
- 用Pipe操作符组合处理流程
- 实现错误处理和数据验证
- 支持流式输出

In [10]:
# 实战：智能摘要生成器

@chain_decorator
def document_preprocessor(input_data):
    """文档预处理器"""
    text = input_data.get("document", "").strip()
    
    if not text:
        raise ValueError("文档内容不能为空")
    
    if len(text) < 50:
        raise ValueError("文档内容太短，至少需要50个字符")
    
    # 清理文本
    cleaned = text.replace("\n\n", "\n").replace("\t", " ")
    words = cleaned.split()
    
    return {
        "original_text": text,
        "cleaned_text": cleaned,
        "word_count": len(words),
        "char_count": len(cleaned),
        "estimated_reading_time": len(words) // 200  # 假设每分钟读200字
    }

@chain_decorator
def create_summary_prompt(doc_data):
    """创建摘要提示词"""
    text = doc_data["cleaned_text"]
    word_count = doc_data["word_count"]
    
    # 根据文档长度调整摘要要求
    if word_count < 200:
        summary_length = "2-3句话"
    elif word_count < 500:
        summary_length = "一个段落（4-5句话）"
    else:
        summary_length = "2-3个段落"
    
    prompt_template = ChatPromptTemplate.from_template("""
请为以下文档生成摘要：

文档内容：
{text}

要求：
1. 摘要长度：{summary_length}
2. 抓住核心要点和主要观点
3. 保持客观和准确
4. 使用清晰简洁的语言

请生成摘要：""")
    
    return prompt_template.format_messages(
        text=text,
        summary_length=summary_length
    )

@chain_decorator
def format_summary_report(ai_response):
    """格式化摘要报告"""
    summary = ai_response.content
    
    report = f"""
📄 智能摘要报告
{'=' * 40}

📝 摘要内容：
{summary}

⏰ 生成时间：2024-01-15 10:30:00
✅ 状态：摘要生成完成

📊 统计信息将在处理完成后添加...
"""
    return report.strip()

# 创建摘要生成Chain
summary_chain = (
    document_preprocessor 
    | create_summary_prompt 
    | model 
    | format_summary_report
)

print("【智能摘要生成器】")

# 测试文档
test_document = """
人工智能（Artificial Intelligence，AI）是计算机科学的一个分支，旨在创建能够执行通常需要人类智能才能完成的任务的系统。AI的发展可以追溯到20世纪50年代，当时研究人员开始探索让机器模拟人类思维的可能性。

现代AI技术包括机器学习、深度学习、自然语言处理、计算机视觉等多个领域。机器学习允许系统从数据中学习并改进性能，而无需明确编程。深度学习是机器学习的一个子集，使用神经网络来处理复杂的数据模式。

AI技术已经广泛应用于各个行业，包括医疗健康、金融服务、交通运输、娱乐媒体等。在医疗领域，AI可以帮助诊断疾病、发现新药；在金融领域，AI用于风险评估和欺诈检测；在交通领域，自动驾驶汽车正在成为现实。

尽管AI带来了许多好处，但也存在一些挑战和担忧，包括就业影响、隐私保护、算法偏见等问题。因此，负责任的AI发展变得越来越重要，需要在技术进步和社会责任之间找到平衡。
"""

try:
    summary_result = summary_chain.invoke({"document": test_document})
    print(summary_result)
    
except Exception as e:
    print(f"摘要生成失败: {e}")

print("\n" + "=" * 60)

# 测试流式输出
print("\n【流式摘要生成】")
print("正在生成摘要...")
print("-" * 40)

for chunk in summary_chain.stream({"document": test_document}):
    print(chunk, end="", flush=True)
print()

【智能摘要生成器】
📄 智能摘要报告

📝 摘要内容：
人工智能（AI）是计算机科学的一个重要分支，旨在创建能执行通常需要人类智能的任务的系统。现代AI技术，如机器学习和深度学习，已在医疗、金融和交通等多个行业得到广泛应用，尽管其发展带来了显著的好处，但也引发了就业、隐私和算法偏见等挑战，因此需要在技术进步与社会责任之间寻求平衡。

⏰ 生成时间：2024-01-15 10:30:00
✅ 状态：摘要生成完成

📊 统计信息将在处理完成后添加...


【流式摘要生成】
正在生成摘要...
----------------------------------------
📄 智能摘要报告

📝 摘要内容：
人工智能（AI）是计算机科学的一个分支，旨在使机器能够执行需要人类智能的任务。现代AI技术涵盖机器学习、深度学习和自然语言处理等领域，已广泛应用于医疗、金融和交通等行业。然而，AI的发展也面临就业影响、隐私保护和算法偏见等挑战，因此负责任的AI发展显得尤为重要。

⏰ 生成时间：2024-01-15 10:30:00
✅ 状态：摘要生成完成

📊 统计信息将在处理完成后添加...


## 本章总结

### 核心知识点

**LCEL基础概念**
- LCEL是LangChain的核心语法
- 所有组件都实现Runnable接口
- 统一的invoke、stream、batch方法

**Pipe操作符（|）**
- 用于连接和组合Runnable组件
- 数据从左到右流动
- 自动处理类型适配

**@chain装饰器**
- 将普通函数转换为Runnable
- 支持所有标准Runnable方法
- 可以与其他组件组合

**RunnableLambda**
- 函数式编程支持
- @chain的底层实现
- 提供灵活的自定义处理

**复杂Chain构建**
- 单一职责原则
- 可组合和可复用设计
- 错误处理和调试策略

### 与前两章的关系

```
第01章：Chat Models基础
  ↓ 学会模型调用
第02章：Prompts & Messages  
  ↓ 学会提示词设计
第03章：LCEL基础
  ↓ 学会组件组合（当前）
第04章：Structured Output
```

### 重要提醒

**LCEL的威力**
- 简洁的语法表达复杂逻辑
- 自动支持流式和批量处理
- 强大的可组合性

**最佳实践**
- 保持组件简单和专一
- 合理的错误处理
- 清晰的数据流设计
- 充分测试每个组件

### 练习建议

1. 尝试用@chain创建不同类型的处理函数
2. 构建包含3-5个组件的复杂Chain
3. 实验流式输出在实际场景中的应用
4. 练习错误处理和调试技巧